In [1]:
import orientationpy
import numpy
import tifffile
import napari
import matplotlib
import numpy as np
import napari.layers
from napari.types import LayerDataTuple
from magicgui import magicgui
import matplotlib.pyplot as plt

In [2]:
# Triangles

image = tifffile.imread('/home/wittwer/data/triangles.tif')
image = image.astype(float) / 256
rx, ry = image.shape

image.shape

(1302, 1470)

In [ ]:
# Sam's data

image = tifffile.imread('/home/wittwer/data/sam/sam-197081.tif')[12]
image = image.astype(float) / 256
rx, ry = image.shape

image.shape

In [ ]:
# Orientationpy data

image = tifffile.imread('/home/wittwer/code/clones/orientationpy/data/2D/image1_from_OrientationJ.tif')
image = image.astype(float) / 256
rx, ry = image.shape

image.shape

In [ ]:
# Orientationpy data

image = tifffile.imread('/home/wittwer/code/clones/orientationpy/data/2D/chirp2.tif')
image = image.astype(float) / 256
rx, ry = image.shape

image.shape

In [ ]:
# Spiral

image = tifffile.imread('/home/wittwer/code/clones/orientationpy/data/2D/BM-spiral.tif')
image = image.astype(float) / 256
rx, ry = image.shape

image.shape

### Napari viewer

In [4]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(image, interpolation='nearest')

# mask = (image < 0.05).astype(int)
# viewer.add_image(mask, blending='additive', colormap='gray')

<Image layer 'image' at 0x7fbe673b59a0>

## Orientation comptuation

In [ ]:
# gx, gy = orientationpy.computeGradient(image, mode='gaussian')

# structureTensor = orientationpy.computeStructureTensor((gx, gy), sigma=2)

# orientations = orientationpy.computeOrientation(
#     structureTensor, 
#     computeEnergy=True, 
#     computeCoherency=True
# )

# theta = orientations['theta']
# energy = orientations['energy']
# coherency = orientations['coherency']

### Matplotlib viz

In [ ]:
# # Should this be in the orientation computation function?
# coherency[numpy.isnan(coherency)] = 0
# coherency[image == 0] = 0
# ##############

# imDisplayHSV = numpy.zeros((image.shape[0], image.shape[1], 3), dtype='f4')

# # Hue is the orientation (nice circular mapping)
# imDisplayHSV[:, :, 0] = (theta + 90) / 180

# # Saturation is coherency
# imDisplayHSV[:, :, 1] = coherency / coherency.max()

# # Value is original image ;)
# imDisplayHSV[:, :, 2] = image / image.max()

# fig, ax = plt.subplots()
# cmap = matplotlib.cm.hsv
# norm = matplotlib.colors.Normalize(vmin=-90, vmax=90)
# fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap))
# ax.imshow(matplotlib.colors.hsv_to_rgb(imDisplayHSV))
# plt.show()

In [ ]:
# ### Option 2

# alpha = coherency / (2 * coherency.max())

# plt.imshow(image, cmap="Greys_r", vmin=0)
# plt.imshow(theta, cmap="hsv", 
#     alpha=alpha,
#     vmin=-90, vmax=90
# )
# plt.show()

### Napari dock widget

In [ ]:
# def compute_orientation(image: np.ndarray, sigma: float) -> np.ndarray:

#     gx, gy = orientationpy.computeGradient(image, mode='gaussian')

#     structureTensor = orientationpy.computeStructureTensor((gx, gy), sigma=sigma)

#     orientations, energy, coherency = orientationpy.computeOrientation(
#         structureTensor, 
#         computeEnergy=True, 
#         computeCoherency=True
#     )

#     return orientations

# @magicgui(call_button='Run', sigma={'widget_type': 'FloatSpinBox', 'min': 0, 'step': 0.1})
# def orientation_widget(image_layer: napari.layers.Image, sigma: float=1.0) -> LayerDataTuple:
#     if image_layer is None:
#         return

#     image = image_layer.data

#     orientations = compute_orientation(image, sigma=sigma)

#     return (orientations, {
#         'name': 'orientation', 
#         'colormap': 'hsv', 
#         'blending': 'additive', 
#         'opacity': 0.4
#         }, 'image')

# viewer.window.add_dock_widget(orientation_widget, name='Orientationpy')

## Boxes

In [5]:
def compute_vectors(image, boxSizePixels=5):

    structureTensorBoxes = orientationpy.computeGradientStructureTensorBoxes(
        image,
        [boxSizePixels, boxSizePixels],
    )

    orientations = orientationpy.computeOrientation(
        structureTensorBoxes,
        mode="fiber",
        computeEnergy=True,
        computeCoherency=True,
    )

    orientationBoxes = orientations['theta']
    energyBoxes = orientations['energy']

    boxVectorsYX = orientationpy.anglesToVectors(orientations)

    boxCentresX, boxCentresY = numpy.mgrid[
        boxSizePixels // 2 : orientationBoxes.shape[0] * boxSizePixels + boxSizePixels // 2 : boxSizePixels,
        boxSizePixels // 2 : orientationBoxes.shape[1] * boxSizePixels + boxSizePixels // 2 : boxSizePixels,
    ]

    boxCentres = np.concatenate((boxCentresX[None], boxCentresY[None]), axis=0)

    _, bx, by = boxCentres.shape

    bc = boxCentres.reshape((2, bx*by))

    bv = boxVectorsYX.reshape((2, bx*by))

    # bv *= energyBoxes.reshape((bx*by)) / energyBoxes.max()
    bv *= boxSizePixels

    vectors = np.concatenate((bc[None], bv[None]), axis=0)
    vectors = np.rollaxis(vectors, axis=2)

    return vectors, orientationBoxes


@magicgui(call_button='Run', box_size={'widget_type': 'SpinBox', 'min': 1})
def vectors_widget(image_layer: napari.layers.Image, box_size: int=3) -> LayerDataTuple:
    if image_layer is None:
        return

    image = image_layer.data

    vectors, orientations = compute_vectors(image, boxSizePixels=box_size)
    print(orientations.shape)
    oris = orientations.ravel()
    oris += 90
    oris /= 180
    print(oris.min(), oris.max())

    return (vectors, {
        'name': 'vectors', 
        'edge_color': 'orientation', 
        'edge_width': 0.7,
        'features': {'orientation': oris}
        }, 'vectors')

viewer.window.add_dock_widget(vectors_widget, name='Orientation vectors')

100%|██████████| 186/186 [00:00<00:00, 298.99it/s]


(186, 210)
6.873749965071359e-06 0.9999875996859503


### Matplotlib quiver

In [ ]:
plt.imshow(image, cmap="Greys_r", vmin=0)
plt.quiver(
    boxCentresX,
    boxCentresY,
    boxVectorsYX[1],
    boxVectorsYX[0],
    angles="xy",
    scale_units="xy",
    # scale=energyBoxes.ravel(),
    color="r",
    headwidth=0,
    headlength=0,
    headaxislength=1,
)
plt.show()